In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix, csc_matrix
from signet.cluster import Cluster
import glob

In [5]:
# Path to the 'Yearly' directory
data_root = "Data/CRSP/Yearly"
START_YEAR = 2000
END_YEAR = 2015

In [4]:
def load_gzipped_data(data_root):
    """
    Loads all .csv.gz daily files from Yearly/2000 through Yearly/2015.
    Extracts the date from the filename and adds it as a column.
    """
    all_files = []
    for year in range(START_YEAR, END_YEAR + 1):
        pattern = os.path.join(data_root, str(year), "*.csv.gz")
        files = sorted(glob.glob(pattern))
        
        for f in files:
            try:
                df_day = pd.read_csv(f, compression='gzip')
                date_str = os.path.basename(f).replace(".csv.gz", "")
                df_day["date"] = pd.to_datetime(date_str, format="%Y%m%d")
                all_files.append(df_day)
            except Exception as e:
                print(f"Skipping {f}: {e}")

    df = pd.concat(all_files, ignore_index=True)
    return df

In [5]:
df = load_gzipped_data("Data/CRSP/Yearly")

Note -- in sponge paper they eventaully subtract off an R_SPY term from the daily return that more or less regularizes things. eventually we should do this <br>
also make sure that its not making Nan values on either side of the temporal window

In [6]:
def compute_daily_return_matrix(df, window_dates):
    """
    Computes a matrix of daily log returns within a given window.

    Returns:
    - DataFrame with tickers as rows and dates as columns
    """
    df = df.copy()
    df = df[df['date'].isin(window_dates)]
    df = df[df['prevAdjClose'] > 0]

    df = df.sort_values(['ticker', 'date'])
    df['log_price'] = np.log(df['prevAdjClose'])

    # Compute daily returns 
    df['log_return'] = df.groupby('ticker')['log_price'].diff()

    df = df.dropna(subset=['log_return']) ##CHECK THIS -- Would first day be 0 because of the way this is done? might need an extra day in windowdates array (note: i think this is fixed)

    #I think(?) this solves the problem of the first day having an NaN value
    actual_window_dates = sorted(window_dates)[1:]
    df = df[df['date'].isin(actual_window_dates)]

    ##TODO: substract off the R_SPY term

    return df.pivot(index='ticker', columns='date', values='log_return')

Check on this: verify that sponge clustering code works with WEIGHTED adjacency matrix, not just binary adjacency matrix. if its not weighted then i dont see how weights come in?

In [7]:
def build_signed_graph(R, threshold=0.3):
    corr = R.T.corr().fillna(0)
    np.fill_diagonal(corr.values, 0)
    Ap = csc_matrix(corr.values * (corr.values >= threshold))
    An = csc_matrix(-corr.values * (corr.values <= -threshold))
    return Ap, An

In [8]:
def sponge_clustering(Ap, An, k=5, tau_p=1, tau_n=1):
    clusterer = Cluster((Ap, An))
    labels = clusterer.SPONGE(k=k, tau_p=tau_p, tau_n=tau_n)
    return labels

In [9]:
## Step 2 of the pipeline -- to be called during run the rolling sponge pipleine

def compute_synthetic_etfs(R, clustering_result):
    """
    Create synthetic ETFs for each cluster on a given date.
    
    Parameters:
    - R: pandas DataFrame (tickers × dates) of returns for the current window
    - clustering_result: dict with 'date' and 'clusters' from run_rolling_sponge_clustering
    
    Returns:
    - dict mapping cluster ID ([k]) to synthetic ETF time series
    """
    etfs = {}
    date = clustering_result['date']
    clusters = clustering_result['clusters']

    for label, tickers in clusters.items():
       # if len(tickers) < 2: ##might not be necessary
       #     continue  # Skip singleton clusters (ETF is trivial or meaningless)

        # Submatrix of returns for the cluster
        cluster_returns = R.loc[tickers]

        # Each row is an asset, columns are time. compute centroid as numpy array for broadcasting
        centroid = cluster_returns.mean(axis=0, skipna=True)
        centroid = centroid.values

        # Euclidean distance of each asset to the centroid
       # distances = np.linalg.norm(cluster_returns.values - centroid.values, axis=1)
        #The diffs line is to treat the NANs as 0s just for the sake of distance calculation. This could use work because data we dont have is being treated as "important" i.e. close to center 
        #TODO Fix the way this is handled -- I don't like that an asset with a lot of missing data will have a higher weight than other assets
        diffs = np.nan_to_num(cluster_returns.values - centroid)
        distances = np.linalg.norm(diffs, axis=1)       

        # Convert to weights: inverse distance, avoid div by 0. less distance to center implies stronger weights.
        epsilon = 1e-8
        inv_distances = 1 / (distances + epsilon)
        weights = inv_distances / inv_distances.sum() #normalize

        # Weighted average across assets → synthetic ETF
        #synthetic_etf = np.average(cluster_returns.values, axis=0, weights=weights)
        ##converts NaN values to 0 so they won't impact the overall value of the average return
        synthetic_etf = np.average(np.nan_to_num(cluster_returns.values), axis=0, weights=weights)

        etfs[label] = pd.Series(synthetic_etf, index=cluster_returns.columns)


    return {
        'date': date,
        'synthetic_etfs': etfs
    }

The below is good for performing rolling sponge clustering to obtain all k x T dataframes at once. The output is a dictionary, but can be thought of as an N x k x T tensor where N is the total number of sliding windows we consider. Note that the clusters at each window as we iterate over N may be different

In [10]:
# def run_rolling_sponge_clustering(df, lookback=20, threshold=0.3, k=5):
#     df = df.sort_values('date')
#     unique_dates = df['date'].drop_duplicates().sort_values().reset_index(drop=True)

#     results = []

#     for i in range(lookback, len(unique_dates)):
#         window_dates = unique_dates[i - lookback - 1:i] ##minus 1 because we need to make sure we go back an extra day so that the first day of our lookback window has the amount of time it needs
#         R = compute_daily_return_matrix(df, window_dates)
#         tickers = R.index.tolist()
        
#         if R.shape[1] < 2:
#             # Not enough data to compute correlation
#             continue

#         Ap, An = build_signed_graph(R, threshold=threshold)
#         labels = sponge_clustering(Ap, An, k=k)

#         # Build cluster dictionary
#         cluster_dict = {}
#         for ticker, label in zip(tickers, labels):
#             cluster_dict.setdefault(label, []).append(ticker)

#         clustering_result = {
#             'date': unique_dates[i],
#             'clusters': cluster_dict
#         }

#         synthetic_eft_clustering_results = compute_synthetic_etfs(R, clustering_result)

#         results.append(synthetic_eft_clustering_results)

#         print(f"Processed clustering and ETF synthesis for window ending {unique_dates[i].date()}")

#     return results

In [23]:
def run_sponge_clustering_window(df, start_date, end_date, threshold=0.3, k=5):
    # Ensure datetime format
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')

    # Drop duplicates and sort unique dates
    unique_dates = df['date'].drop_duplicates().sort_values().reset_index(drop=True)

    # Find the index of start_date and go one day earlier
    start_idx = unique_dates[unique_dates >= pd.to_datetime(start_date)].index[0]
    adjusted_start_date = unique_dates[max(0, start_idx - 1)]

    # Filter data between the two dates
    #mask = (df['date'] >= adjusted_start_date) & (df['date'] <= pd.to_datetime(end_date))
    #window_df = df[mask]

    # Extract unique dates in the window (sorted)
    window_dates = unique_dates[(unique_dates >= adjusted_start_date) & (unique_dates <= pd.to_datetime(end_date))].reset_index(drop=True)


    # Compute the return matrix for the entire window
    R = compute_daily_return_matrix(df, window_dates)
    tickers = R.index.tolist()

    if R.shape[1] < 2:
        raise ValueError("Not enough time points to compute return correlations.")

    # Build signed graph and run clustering
    Ap, An = build_signed_graph(R, threshold=threshold)
    labels = sponge_clustering(Ap, An, k=k)

    # Build cluster dictionary
    cluster_dict = {}
    for ticker, label in zip(tickers, labels):
        cluster_dict.setdefault(label, []).append(ticker)

    clustering_result = {
        'date': pd.to_datetime(end_date),  # Or just a placeholder
        'clusters': cluster_dict
    }

    synthetic_result = compute_synthetic_etfs(R, clustering_result)
    etfs = synthetic_result['synthetic_etfs']  # dict: cluster_id -> pd.Series

    # Build and return k × T DataFrame
    etf_df = pd.DataFrame.from_dict(etfs, orient='index')  # clusters as rows
    return etf_df


In [26]:
test_result = run_sponge_clustering_window(df, '05-31-2000', '07-25-2000')

In [27]:
test_result

,2000-05-31,2000-06-01,2000-06-02,2000-06-05,2000-06-06,2000-06-07,2000-06-08,2000-06-09,2000-06-12,2000-06-13,...,2000-07-12,2000-07-13,2000-07-14,2000-07-17,2000-07-18,2000-07-19,2000-07-20,2000-07-21,2000-07-24,2000-07-25
1,0.001629,0.001626,0.002398,0.000788,0.003410,0.002124,0.002050,0.000762,0.002827,-0.000172,...,0.000072,0.000990,-0.000116,0.001379,0.001228,0.000025,-0.000786,0.000177,-0.000223,-0.000604
3,0.005842,0.005495,0.013021,0.029125,-0.004622,-0.007103,0.003770,-0.005561,0.001034,-0.002866,...,0.003629,0.003633,-0.001187,0.003886,-0.006408,-0.002998,-0.004125,0.005052,-0.001942,-0.004260
2,0.007235,-0.000992,0.008194,0.005874,-0.010585,0.001383,0.000707,-0.005106,0.003820,0.000308,...,0.002348,0.004068,0.000056,-0.000513,-0.000919,-0.004051,-0.001593,0.005316,-0.006253,-0.005128
0,0.035043,-0.001741,0.020940,0.028434,-0.002881,-0.003495,0.004504,-0.004172,0.004914,-0.011433,...,-0.001790,0.011732,0.002311,0.004773,0.003053,-0.012701,-0.009968,0.009435,-0.012216,-0.010631
4,0.011576,0.029993,-0.000238,-0.033174,-0.023042,0.028894,-0.004207,0.009973,-0.005656,0.024532,...,0.040200,-0.009611,-0.007965,-0.000427,-0.011594,0.009806,-0.008414,-0.007703,-0.022610,-0.030935


ANNIE: Below is the function you can eventually use to do your part. It takes in an overall start date, overall end date and window size. It calls the run_sponge_clustering_window() function that JUST computes the k x T dataframe for a window of size T. It iteratively does this process from the overall start date to the overall end date with a step size of T (window_size) until the end date is reached. You can add the code you want to call for each k x T dataframe into the try block. Feel free the change the function name too

In [28]:
def run_rolling_sponge_clustering(df, overall_start_date, overall_end_date, window_size, threshold=0.3, k=5):
    """
    Run SPONGE clustering in a rolling window fashion over a given date range.

    Returns:
    - results: list of tuples (window_start_date, k × T DataFrame)
    """
    # Ensure datetime format
    df['date'] = pd.to_datetime(df['date'])
    overall_start_date = pd.to_datetime(overall_start_date)
    overall_end_date = pd.to_datetime(overall_end_date)

    # Filter trading dates between overall_start and overall_end
    all_dates = df['date'].drop_duplicates().sort_values().reset_index(drop=True)
    unique_dates = all_dates[(all_dates >= overall_start_date) & (all_dates <= overall_end_date)].reset_index(drop=True)

    results = []

    # Iterate through valid rolling windows
    for i in range(len(unique_dates) - window_size + 1):
        window_start = unique_dates[i]
        window_end = unique_dates[i + window_size - 1]

        try:
            etf_df = run_sponge_clustering_window(
                df,
                start_date=window_start,
                end_date=window_end,
                threshold=threshold,
                k=k
            )
            results.append((window_start, etf_df))
            print(f"Processed window: {window_start.date()} to {window_end.date()}")
        except Exception as e:
            print(f"Skipping window {window_start.date()} to {window_end.date()} due to error: {e}")
            continue

    return results


In [30]:
results = run_rolling_sponge_clustering(df, '05-31-2000', '08-01-2000', 20)

Processed window: 2000-05-31 to 2000-06-27
Processed window: 2000-06-01 to 2000-06-28
Processed window: 2000-06-02 to 2000-06-29
Processed window: 2000-06-05 to 2000-06-30
Processed window: 2000-06-06 to 2000-07-03
Processed window: 2000-06-07 to 2000-07-05
Processed window: 2000-06-08 to 2000-07-06
Processed window: 2000-06-09 to 2000-07-07
Processed window: 2000-06-12 to 2000-07-10
Processed window: 2000-06-13 to 2000-07-11
Processed window: 2000-06-14 to 2000-07-12
Processed window: 2000-06-15 to 2000-07-13
Processed window: 2000-06-16 to 2000-07-14
Processed window: 2000-06-19 to 2000-07-17
Processed window: 2000-06-20 to 2000-07-18
Processed window: 2000-06-21 to 2000-07-19
Processed window: 2000-06-22 to 2000-07-20
Processed window: 2000-06-23 to 2000-07-21
Processed window: 2000-06-26 to 2000-07-24
Processed window: 2000-06-27 to 2000-07-25
Processed window: 2000-06-28 to 2000-07-26
Processed window: 2000-06-29 to 2000-07-27
Processed window: 2000-06-30 to 2000-07-28
Processed w

In [107]:
#df = your_loaded_dataframe_with_all_data
results = run_rolling_sponge_clustering(df, lookback=20, threshold=0.3, k=5)

Processed clustering and ETF synthesis for window ending 2000-02-02
Processed clustering and ETF synthesis for window ending 2000-02-03
Processed clustering and ETF synthesis for window ending 2000-02-04
Processed clustering and ETF synthesis for window ending 2000-02-07
Processed clustering and ETF synthesis for window ending 2000-02-08
Processed clustering and ETF synthesis for window ending 2000-02-09
Processed clustering and ETF synthesis for window ending 2000-02-10
Processed clustering and ETF synthesis for window ending 2000-02-11
Processed clustering and ETF synthesis for window ending 2000-02-14
Processed clustering and ETF synthesis for window ending 2000-02-15
Processed clustering and ETF synthesis for window ending 2000-02-16
Processed clustering and ETF synthesis for window ending 2000-02-17
Processed clustering and ETF synthesis for window ending 2000-02-18
Processed clustering and ETF synthesis for window ending 2000-02-22
Processed clustering and ETF synthesis for windo

In [108]:
results

[{'date': Timestamp('2000-02-02 00:00:00'),
  'synthetic_etfs': {1: date
   2000-01-04   -0.027090
   2000-01-05   -0.016000
   2000-01-06    0.014421
   2000-01-07    0.015999
   2000-01-10    0.017367
   2000-01-11   -0.005227
   2000-01-12   -0.010415
   2000-01-13    0.000153
   2000-01-14    0.008664
   2000-01-18   -0.000276
   2000-01-19   -0.012027
   2000-01-20   -0.001655
   2000-01-21   -0.011328
   2000-01-24   -0.005630
   2000-01-25   -0.015130
   2000-01-26   -0.007428
   2000-01-27    0.010287
   2000-01-28    0.000636
   2000-01-31   -0.013315
   2000-02-01    0.010319
   dtype: float64,
   3: date
   2000-01-04    0.015221
   2000-01-05    0.001077
   2000-01-06    0.001177
   2000-01-07    0.001178
   2000-01-10    0.001141
   2000-01-11    0.000144
   2000-01-12   -0.005903
   2000-01-13   -0.001576
   2000-01-14   -0.002519
   2000-01-18   -0.002160
   2000-01-19   -0.002028
   2000-01-20   -0.003115
   2000-01-21   -0.001811
   2000-01-24   -0.003365
   2000-01-25

In [120]:
# Let's assume `results` is your list as described.
results_by_date = {}

for entry in results:
    date = entry['date']
    synthetic_etfs = entry['synthetic_etfs']

    # Create a DataFrame from the cluster dictionary
    df = pd.DataFrame.from_dict(synthetic_etfs, orient='index')

    # Store the DataFrame using the date as a key
    results_by_date[date] = df

In [128]:
results_by_date[pd.Timestamp('2000-05-31')] 

,2000-05-02,2000-05-03,2000-05-04,2000-05-05,2000-05-08,2000-05-09,2000-05-10,2000-05-11,2000-05-12,2000-05-15,2000-05-16,2000-05-17,2000-05-18,2000-05-19,2000-05-22,2000-05-23,2000-05-24,2000-05-25,2000-05-26,2000-05-30
2,0.005807,0.017419,-0.026081,0.010637,0.004036,0.003477,-0.000148,-0.008383,0.006183,0.004158,0.009951,-0.002642,-0.003356,-0.002401,-0.006731,-0.010860,0.003334,-0.015347,-0.001658,-0.003593
1,-0.003306,-0.002643,0.005185,-0.003132,-0.001117,-0.002232,-0.000023,-0.001248,-0.003249,-0.000583,-0.003741,0.000532,0.000906,-0.001163,0.001188,-0.001932,-0.001141,-0.001781,0.001392,-0.001936
3,0.004770,-0.014425,-0.019246,0.006443,0.018897,-0.011615,-0.005247,-0.023969,0.019023,0.013798,0.005556,0.014136,-0.013549,-0.005229,-0.019580,-0.014512,-0.009558,0.001402,-0.001265,-0.004380
0,0.004336,-0.009185,-0.008503,0.006471,-0.000951,0.004815,-0.001247,-0.003837,0.017474,-0.005033,0.017940,-0.004770,-0.013257,0.002739,-0.009200,0.007825,-0.006422,0.016228,-0.018539,0.007197
4,0.040103,-0.010220,-0.008248,0.003373,-0.000354,-0.004627,-0.005716,-0.016063,0.009098,-0.000901,0.001386,0.003647,-0.002390,-0.006770,-0.007430,-0.008042,-0.002234,-0.007229,-0.004179,0.000661
